In [329]:
import numpy as np
import pandas as pd
import scipy.io

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data = scipy.io.loadmat('qm7.mat')
X = data ['X']
y = data['T']
R = data['R']
P = data['P']


In [27]:
np.shape(data['X'][0])

(23, 23)

In [30]:
np.shape(data['X'][0].flatten())

(529,)

In [15]:
data['X'][1][0]

array([36.858105 , 12.599944 ,  2.9019997,  1.4731182,  1.4731014,
        2.9019725,  2.9018855,  1.4731016,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ], dtype=float32)

In [33]:
data['T'][0][0]

-417.96

In [32]:
data['R'][0]

array([[ 1.886438  , -0.00464873, -0.00823921],
       [ 3.9499245 , -0.00459203,  0.00782347],
       [ 1.1976895 ,  1.9404842 ,  0.00782347],
       [ 1.1849339 , -0.99726516,  1.6593875 ],
       [ 1.2119948 , -0.9589793 , -1.710958  ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.   

In [140]:
triuind = (np.arange(23)[:,np.newaxis] <= np.arange(23)[np.newaxis,:]).flatten()

In [332]:
def realize(X):
		def _realize_(x):
			inds = np.argsort(-(x**2).sum(axis=0)**.5)
			x = x[inds,:][:,inds]*1
			x = x.flatten()[triuind]
			return x
		return np.array([_realize_(z) for z in X])
    
def normalize(X): return X/X.max()

In [334]:
rmse = []
mae = []

for split in range(1, 5):
    Ptrain = data['P'][list(range(0, split)) + list(range(split+1, 5))].flatten()
    Ptest  = data['P'][split]
    
    X_train = data['X'][Ptrain]
    y_train = data['T'][0, Ptrain]
#     sns.distplot(y_train)
#     plt.show()
    
#     y_train = y_train/-2192.0

    X_test = data['X'][Ptest]
    y_test = data['T'][0, Ptest]
#     y_test = y_test/-2192.0
    
#     sns.distplot(y_train)
#     plt.show()
    
    X_train_sorted = normalize(realize(X_train))
    X_test_sorted = normalize(realize(X_test))
    regr = linear_model.LinearRegression()

    regr.fit(X_train_sorted, y_train)

    y_pred = regr.predict(X_test_sorted)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))

RMSE list [27.708935, 26.857279, 27.013254, 26.137651]
Rmse 26.92928
MAE list [20.810917, 21.001122, 20.828333, 20.087189]
MAE 20.68189


In [417]:
from sklearn.model_selection import KFold

rmse = []
mae = []
regr = LinearRegression()

X = data ['X']
X = normalize(realize(X))
y = data['T'][0]

P = data['P']

# print(P)

kf = KFold(n_splits=5, shuffle=True, random_state=234)

train_indices = []
test_indices = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
#         print(fold, "Train:", train_idx, "Test:", val_idx)
        train_indices.append(train_idx)
        test_indices.append(val_idx)
print(train_indices)
print(test_indices)
for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))

[array([   0,    1,    2, ..., 7162, 7163, 7164]), array([   0,    3,    4, ..., 7162, 7163, 7164]), array([   0,    1,    2, ..., 7162, 7163, 7164]), array([   0,    1,    2, ..., 7159, 7161, 7162]), array([   1,    2,    3, ..., 7160, 7163, 7164])]
[array([   4,    5,    7, ..., 7134, 7142, 7153]), array([   1,    2,    8, ..., 7149, 7150, 7156]), array([   9,   14,   16, ..., 7137, 7152, 7159]), array([   3,    6,   25, ..., 7160, 7163, 7164]), array([   0,   11,   13, ..., 7155, 7161, 7162])]


In [514]:
import joblib

In [570]:
def test_base_models(models, X, y):
    results = {}
    for i in models:
        rmse = []
        mae = []
        for train_index, test_index in kf.split(X, y):
#             print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            y_pred = models[i].fit(X_train, y_train).predict(X_test)
#             y_pred = regr.predict(X_test)

            rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
            rmse.append(rmse_values)

            mae_values = mean_absolute_error(y_test, y_pred)
            mae.append(mae_values)
        results[i] = [np.mean(rmse), np.mean(mae), models[i].best_params_]
        results = pd.DataFrame(results)
    
    
    joblib.dump(models, 'blah.pkl') 
    results.insert(loc=0, column='metrics', value=['RMSE', 'MAE', 'best_params'])
    return results

In [516]:
X = data ['X']
X = normalize(realize(X))
y = data['T'][0]

linear_models = {
#     'OLS': linear_model.LinearRegression(),
         'Lasso': linear_model.Lasso(),
         'Ridge': linear_model.Ridge(),}

In [517]:
test_base_models(linear_models, X, y)

AttributeError: 'Lasso' object has no attribute 'best_params_'

In [567]:
lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

models2 = {
#     'OLS': linear_model.LinearRegression(),
           'Lasso': GridSearchCV(linear_model.Lasso(), cv=5,
                               param_grid=lasso_params),
           'Ridge': GridSearchCV(linear_model.Ridge(), cv =5,
                               param_grid=ridge_params)}


In [568]:
kf = KFold(n_splits=5, shuffle=False, random_state=234)
test_base_models(models2, X, y)

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 

,metrics,Lasso,Ridge
0,RMSE,39.2542,64.4987
1,MAE,28.6846,48.2588
2,best_params,{'alpha': 0.02},{'alpha': 200}


In [539]:
!ls

blah.pkl  preprocessdata.py  regression_models.py
data	  qm7.mat	     savedata.py
input	  qm7-mlp.ipynb      Untitled.ipynb


In [569]:
some_model = joblib.load('blah.pkl')  
some_model

{'Lasso': GridSearchCV(cv=5, error_score='raise',
        estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
        fit_params=None, iid=True, n_jobs=1,
        param_grid={'alpha': [0.02, 0.024, 0.025, 0.026, 0.03]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
        scoring=None, verbose=0),
 'Ridge': GridSearchCV(cv=5, error_score='raise',
        estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
        fit_params=None, iid=True, n_jobs=1,
        param_grid={'alpha': [200, 230, 250, 265, 270, 275, 290, 300, 500]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
        scoring=None, verbose=0)}

In [571]:
some_model = joblib.load('models/regression.pkl')  
some_model

ModuleNotFoundError: No module named 'sklearn.linear_model._coordinate_descent'

In [533]:
rmse = []
mae = []
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#             y_pred = models[i].fit(X_train, y_train).predict(X_test)
    y_pred = some_model.predict(X_test)

    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)

    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)
print(rmse)

[40.186413, 30.796738, 28.967573, 29.617586, 55.506138]


In [418]:
zip(train_indices, test_indices)

In [426]:
for i in test_indices:
    
    print(y[i])
    

[ -808.87  -677.16  -860.33 ... -1766.22 -1628.79 -1507.18]
[ -712.42  -564.21 -1008.49 ... -1475.95 -1189.97 -1661.78]
[ -861.73  -871.19 -1010.86 ... -1802.17 -1789.07 -1919.48]
[ -404.88  -796.98  -914.2  ... -1919.54 -1782.01 -1919.  ]
[ -417.96  -725.93  -618.72 ... -1921.33 -1657.63 -1662.1 ]


In [429]:
np.size(test_indices)

7165

In [398]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.01, gamma = 0.01)

def normalize1(X): return X-X.mean()/X.std()

X = data ['X']
X = normalize1(realize(X))
y = data['T'][0]

kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1, 0.1, 1e-2, 1e-3, 1e-4],
                              "gamma": [1, 2, 10, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-10]})
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kr.fit(X_train, y_train)
    y_pred = kr.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))


print(kr.best_params_)
print(kr.best_score_)

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.089942e-08
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.660147e-08
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.642369e-08
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.305786e-08
  overwrite_a

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.801504e-08
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.189888e-08
  overwrite_a=False)
/home/bis/anac

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/si

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.967743e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.999744e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.971461e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/ho

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.517836e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.742093e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.933097e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.958506e-09
  overwrite_a

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.737927e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.515868e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.692144e-09
  overwrite_a=False)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.525487e-09
  overwrite_a

RMSE list [18.75174, 20.1958, 17.09736, 19.225979, 22.229748]
Rmse 19.500126
MAE list [13.6583185, 14.357466, 13.316499, 14.006273, 13.739338]
MAE 13.815579
{'alpha': 0.001, 'gamma': 1e-05}
0.9391227253210548


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.778746e-08
  overwrite_a=False)


In [547]:
rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.01, gamma = 0.01)

def normalize1(X): return X-X.mean()/X.std()

X = data ['X']
X = normalize1(realize(X))
y = data['T'][0]

kr = GridSearchCV(Ridge(), cv=5,
                  param_grid={"alpha": [1, 0.1, 1e-2, 1e-3, 1e-4],
                              })

ridge = Ridge(alpha=0.1)
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ridge.fit(X_train, y_train)
#     model = kr.best_params_
#     model.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))


# print(kr.best_params_)
# print(kr.best_score_)

RMSE list [27.66101, 27.06842, 26.135584, 27.018282, 27.8817]
Rmse 27.153
MAE list [21.11421, 20.760553, 20.42032, 20.417934, 20.774183]
MAE 20.69744


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.051926e-09
  overwrite_a=True).T
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.151235e-09
  overwrite_a=True).T
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.163623e-09
  overwrite_a=True).T
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.019418e-09
  overwrit

In [553]:
rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.01, gamma = 0.01)

X = data ['X']
X = normalize1(realize(X))
y = data['T'][0]

kr = GridSearchCV(Ridge(), cv=5,
                  param_grid={"alpha": [1, 0.1, 1e-2, 1e-3, 1e-4],
                              })

ridge = Ridge(alpha=0.1)
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

rmse = [np.sqrt(mean_squared_error(
    ridge.fit(X[train_index], y[train_index]).predict(X[test_index]), y[test_index])) for train_index, test_index in kf.split(X, y)]
# for train_index, test_index in kf.split(X, y):
# #     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     ridge.fit(X_train, y_train)
# #     model = kr.best_params_
# #     model.fit(X_train, y_train)
#     y_pred = ridge.predict(X_test)
    
#     rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
#     rmse.append(rmse_values)
    
#     mae_values = mean_absolute_error(y_test, y_pred)
#     mae.append(mae_values)

# print('RMSE list', rmse)
# print('Rmse', np.mean(rmse))
# print('MAE list', mae)
# print('MAE', np.mean(mae))
                
print (rmse)

[27.66101, 27.06842, 26.135584, 27.018282, 27.8817]


In [399]:
kr.cv_results_

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-p

{'mean_fit_time': array([ 1.40048409,  1.40668077,  1.40839133,  1.1832355 ,  1.88280621,
         0.67399478,  0.70026703,  0.63893638,  0.65360913,  1.4788517 ,
         1.40203443,  1.40192223,  1.17653527,  1.68900299,  0.62915092,
         0.62927766,  0.63580823,  0.64111342,  1.39755745,  1.39673352,
         1.39624276,  1.17728362,  1.71425686,  0.63460808,  0.63181624,
         0.63038206,  0.63042331,  1.3992321 ,  1.39594812,  1.39923706,
         1.1801085 ,  1.76345134,  0.63151884,  0.63229179,  0.63477111,
         0.6318275 ,  1.40206776,  1.40253315,  1.39831328,  1.18082175,
         1.75559597,  0.63168035,  0.63480248,  0.63639064, 18.11040201]),
 'std_fit_time': array([3.90016484e-03, 9.47666760e-03, 1.74910446e-02, 2.00172657e-01,
        5.97356505e-01, 3.67324834e-02, 3.55694357e-02, 1.24241872e-02,
        4.59161517e-02, 4.26240461e-02, 9.74531044e-03, 2.27979424e-03,
        1.99120900e-01, 5.12775619e-01, 1.19307891e-03, 1.34875139e-03,
        1.24679027e-

In [400]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, Rando
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.01, gamma = 0.01)

def normalize1(X): return X-X.mean()/X.std()

X = data ['X']
X = normalize1(realize(X))
y = data['T'][0]

kr = GridSearchCV(KernelRidge(kernel='laplacian', gamma=0.1), cv=5,
                  param_grid={"alpha": [1, 0.1, 1e-2, 1e-3, 1e-4],
                              "gamma": [1, 2, 10, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-10]}, n_jobs=-1)
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kr.fit(X_train, y_train)
    y_pred = kr.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))


print(kr.best_params_)
print(kr.best_score_)

RMSE list [6.303950169374139, 7.93390288118707, 5.999396303364735, 6.8662789429524675, 6.461834179644274]
Rmse 6.713072495304539
MAE list [4.11050894451585, 4.40085572677688, 4.1885379185025, 4.311485276630856, 4.225580700139143]
MAE 4.247393713313046
{'alpha': 0.0001, 'gamma': 0.001}
0.9856511101028615


In [401]:
kr.cv_results_

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/bis/anaconda3/lib/python3.6/site-p

{'mean_fit_time': array([ 6.09782104, 10.51099095,  5.12567387,  4.35962558,  4.34938707,
         4.37338901,  4.37339454,  4.3573987 ,  4.36690888,  6.07900224,
        10.50866499,  5.12986507,  4.36358619,  4.35287309,  4.36476331,
         4.37165523,  4.34989924,  4.36059813,  6.1135767 , 10.50441356,
         5.13881969,  4.38184805,  4.35011873,  4.36723223,  4.37040262,
         4.35040145,  4.35788784,  6.07306595, 10.51838932,  5.12961984,
         4.37455559,  4.35112343,  4.36704545,  4.36986265,  4.34924054,
         4.38262806,  6.06802068, 10.51354175,  5.16594563,  4.35430641,
         4.38006301,  4.36929812,  4.36892262,  4.37182207,  4.37109737]),
 'std_fit_time': array([0.64265556, 1.61120749, 0.01563467, 0.02775624, 0.005568  ,
        0.01999758, 0.02430799, 0.01077474, 0.02834696, 0.65889103,
        1.62684496, 0.02425534, 0.02122377, 0.00883478, 0.02815713,
        0.02482612, 0.01051033, 0.02975303, 0.69789049, 1.62415707,
        0.02278817, 0.04477451, 0.00

In [544]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

rmse = []
mae = []
ridge = Ridge()

def normalize1(X): return X-X.mean()/X.std()

X = data ['X']
X = normalize1(realize(X))
y = data['T'][0]

kr = GridSearchCV(Ridge(kernel='polynomial'), cv=5,
                  param_grid={"alpha": [1, 0.1, 1e-2, 1e-3, 1e-4],
                              "gamma": [1, 2, 10, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-10]}, n_jobs=-1)
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kr.fit(X_train, y_train)
    y_pred = kr.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))


print(kr.best_params_)
print(kr.best_score_)

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/si

KeyboardInterrupt: 

In [ ]:
X = np.load('input/X_scaled.npy')
y = np.load('input/y.npy')

lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

models = {'OLS': linear_model.LinearRegression(),
          'Lasso': GridSearchCV(linear_model.Lasso(), 
                                    param_grid=lasso_params),
          'Ridge': GridSearchCV(linear_model.Ridge(), 
                                    param_grid=ridge_params)}

results = test_base_models(models, X, y)
print(results)

In [396]:
rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.001, gamma = 0.01)

def normalize1(X): return X-X.mean()/X.std()

X = data ['X']
X = normalize(realize(X))
y = data['T'][0]

# kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=2,
#                   param_grid={"alpha": [0.1, 1e-2],
#                               "gamma": np.logspace(-2, 2)})
# ridge = Ridge()
# parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 3]}
# ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_error', cv=5)
kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kernelridge.fit(X_train, y_train)
    y_pred = kernelridge.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))


# print(kr.best_params_)
# print(kr.best_score_)

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/bis/anaconda3/lib/python3.6/si

RMSE list [99.25639, 96.41483, 71.16807, 106.31262, 124.38141]
Rmse 99.50667
MAE list [77.96651, 73.66364, 53.500153, 80.49669, 99.1499]
MAE 76.95538


In [ ]:
from sklearn.model_selection import KFold
from sklearn.kernel_ridge import KernelRidge

rmse = []
mae = []
kernelridge = KernelRidge(alpha=0.01, gamma=0.01)

X = data ['X']
X = normalize(realize(X))
y = data['T'][0]


kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kernelridge.fit(X_train, y_train)
    y_pred = kernelridge.predict(X_test)
    
    rmse_values = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse.append(rmse_values)
    
    mae_values = mean_absolute_error(y_test, y_pred)
    mae.append(mae_values)

print('RMSE list', rmse)
print('Rmse', np.mean(rmse))
print('MAE list', mae)
print('MAE', np.mean(mae))

In [229]:
targets = data['T'][0]; targets.shape

(7165,)

In [170]:
from sklearn.kernel_ridge import KernelRidge

In [205]:
from sklearn.model_selection import KFold

scores = []
best_ridge = KernelRidge(alpha=1.0)


kf = KFold(n_splits=5, shuffle=True, random_state=234)

for train_index, test_index in kf.split(inputs):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = inputs[train_index], inputs[test_index]
    y_train, y_test = targets[train_index], targets[test_index]
    best_ridge.fit(X_train, y_train)
    scores.append(best_ridge.score(X_test, y_test))
print (np.mean(scores))

TRAIN: [   0    1    2 ... 7162 7163 7164] TEST: [   4    5    7 ... 7134 7142 7153]


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.656692e-08
  overwrite_a=False)


TRAIN: [   0    3    4 ... 7162 7163 7164] TEST: [   1    2    8 ... 7149 7150 7156]


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.257290e-08
  overwrite_a=False)


TRAIN: [   0    1    2 ... 7162 7163 7164] TEST: [   9   14   16 ... 7137 7152 7159]


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.627222e-08
  overwrite_a=False)


TRAIN: [   0    1    2 ... 7159 7161 7162] TEST: [   3    6   25 ... 7160 7163 7164]


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.618030e-08
  overwrite_a=False)


TRAIN: [   1    2    3 ... 7160 7163 7164] TEST: [   0   11   13 ... 7155 7161 7162]
0.9747492498681789


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:152: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.661141e-08
  overwrite_a=False)


In [206]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
parameters = {'alpha':[1e-15, 1e-10, 1e-5, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='mean_absolute_error', cv=5)
ridge_regressor.fit(inputs, targets)

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/hom

/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/hom

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-15, 1e-10, 1e-05, 0.001, 0.01, 0.1, 1, 5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='mean_absolute_error', verbose=0)

In [207]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 0.1}
-0.018994362466037272


In [208]:
-0.018994362466037272*2192

-41.6356425255537

In [204]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

MSEs = cross_val_score(lr, inputs, targets, scoring='mean_absolute_error', cv=5)
print(np.sqrt(-np.mean(MSEs))*2192.0)
print(np.mean(MSEs)*2192.0)

1412.934798941007
-910.75946444273


/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/bis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/hom

In [196]:
lr.fit(inputs, targets)

array([-3.9383054 , -0.08211157, -0.08237098, -0.08236324,  0.24786419,
        0.24786419, -0.08236866,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.19107655,  0.21416885,
        0.21416904,  0.24786419,  0.24786419,  0.21416889,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.19107655,  0.21416426,  0.24786419,  0.24786419,  0.21416412,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.24786419,  0.24786419,  0.24786419,  0.24786419,
        0.24786419,  0.19107655,  0.24786419,  0.24786419,  0.21

In [39]:
??np.newaxis

In [ ]:
arr = np.arange(1, 10).reshape(3, 3); arr
inds = np.argsort(-(arr**2).sum(axis=0)**.5+np.random.normal(0,1,arr[0].shape)); inds
arr[inds, :]
arr = arr[inds, :][:, inds]; arr
triuind = (np.arange(3)[:,np.newaxis] <= np.arange(3)[np.newaxis,:]).flatten(); triuind
arr.flatten()[triuind]

In [101]:
arr = np.arange(1, 10).reshape(3, 3); arr

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [102]:
(arr**2).sum(axis=0)**.5

array([ 8.1240384 ,  9.64365076, 11.22497216])

In [103]:
inds = np.argsort(-(arr**2).sum(axis=0)**.5+np.random.normal(0,1,arr[0].shape)); inds

array([2, 1, 0])

In [105]:
arr[inds, :]

array([[7, 8, 9],
       [4, 5, 6],
       [1, 2, 3]])

In [110]:
arr = arr[inds, :][:, inds]; arr

array([[9, 8, 7],
       [6, 5, 4],
       [3, 2, 1]])

In [84]:
np.random.normal(0,1,arr[0].shape)

array([ 1.01398054, -1.50826953,  0.42465458])

In [108]:
triuind = (np.arange(3)[:,np.newaxis] <= np.arange(3)[np.newaxis,:]).flatten(); triuind

array([ True,  True,  True, False,  True,  True, False, False,  True])

In [112]:
arr.flatten()[triuind]

array([9, 8, 7, 5, 4, 1])

In [122]:
(np.arange(5)[:,np.newaxis] - np.arange(5)[np.newaxis,:])

array([[ 0, -1, -2, -3, -4],
       [ 1,  0, -1, -2, -3],
       [ 2,  1,  0, -1, -2],
       [ 3,  2,  1,  0, -1],
       [ 4,  3,  2,  1,  0]])

In [123]:
np.arange(5)[:,np.newaxis]

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [128]:
X.shape[1]

23